In [1]:
import os
import sys
import pandas as pd

sys.path.append('/home/fboehning/fboehning')

In [2]:
# IMPORT: labtools
import labtools
from labtools.labhandler import *
from labtools.datahandle import *
from labtools.display_utils import *
from labtools.directory_utils import *
from labtools.dictionary_utils import *

set_display_for_dataframe()
pd.set_option('display.max_rows', 7)

In [3]:
# IMPORT: beesup_llm
# import beesup_llm
# import beesup_llm.toolkit.llm_utils; from beesup_llm.toolkit.llm_utils import *
# import beesup_llm.toolkit.dict_utils; from beesup_llm.toolkit.dict_utils import *
# import beesup_llm.toolkit.setup_utils; from beesup_llm.toolkit.setup_utils import *
# #import beesup_llm.toolkit.visualization; from beesup_llm.toolkit.visualization import *
# #import beesup_llm.toolkit.display_settings; from beesup_llm.toolkit.display_settings import *
# import beesup_llm.toolkit.retrieval_metrics; from beesup_llm.toolkit.retrieval_metrics import *


# import beesup_llm.dataset; from beesup_llm.dataset import *

#import beesup_llm.model_pipelines; from beesup_llm.model_pipelines import *
#import beesup_llm.finetuning_pipelines; from beesup_llm.finetuning_pipelines import *

#import beesup_llm.experiment; from beesup_llm.experiment import *

#import beesup_llm.injection.taxomizer; from beesup_llm.injection.taxomizer import *
#import beesup_llm.injection.rag_pipeline; from beesup_llm.injection.rag_pipeline import *
#import beesup_llm.injection.taxomizer_utils; from beesup_llm.injection.taxomizer_utils import * 
#import beesup_llm.injection.injection_experiment; from beesup_llm.injection.injection_experiment import *

import beesup_llm.llm; from beesup_llm.llm import *
import beesup_llm.emb; from beesup_llm.emb import *

import beesup_llm.extraction; from beesup_llm.extraction import *
import beesup_llm.extraction.extraction_pipeline; from beesup_llm.extraction.extraction_pipeline import *
import beesup_llm.extraction.extraction_experiment; from beesup_llm.extraction.extraction_experiment import *

In [4]:
import importlib
prefixpaths=['labtools','beesup_llm']

def get_modules_with_prefixpath(prefixpaths):
    if isinstance(prefixpaths, str):
        prefixpaths = [prefixpaths]

    modules=dict()
    all_modules = sys.modules.copy()
    for prefixpath in prefixpaths:
        for module_path, module in all_modules.items():
            if module_path.startswith(prefixpath):
                modules[module_path] = module

    return modules

def reimport(prefixpaths=prefixpaths):
    for module_path, module in get_modules_with_prefixpath(prefixpaths).items():
        importlib.reload(sys.modules[module_path])
        globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

LOGGING_FORMAT='%(asctime)s - %(filename)s - %(name)s - %(funcName)s - %(levelname)s - %(message)s'

import logging
logging.basicConfig(
    level=logging.INFO,
    format=LOGGING_FORMAT)

def set_info(prefixpaths=prefixpaths):
    for module_path in get_modules_with_prefixpath(prefixpaths).keys():
        logger = logging.getLogger(module_path)
        logger.setLevel(logging.INFO)
        logging.getLogger().setLevel(logging.INFO)

def set_debug(prefixpaths=prefixpaths):
    for module_path in get_modules_with_prefixpath(prefixpaths).keys():
        logger = logging.getLogger(module_path)
        logger.setLevel(logging.DEBUG)
        logging.getLogger().setLevel(logging.DEBUG)

set_info() 

In [5]:
#LOAD DATA
from datasets import load_from_disk

dataset_ds=load_from_disk('../beede_llm/data/beede_llm_dataset')
dataset_ds

train_df=pd.DataFrame(dataset_ds['train'])
train_df['split']='train'

eval_df=pd.DataFrame(dataset_ds['test'])
eval_df['split']='eval'

dataset_df=pd.concat([train_df,eval_df])
dataset_df

,report_passage,report_scheme,gold_completion,source,is_real,split
0,Beitrag zur Hautflüglerfauna von Bran-\ndenbur...,paragraphs,"```json\n{\n ""meta_scientific_name"": ""Andrena...",https://www.zobodat.at/pdf/Maerkische-Ent-Nach...,True,train
1,"Christoph Saure\n\nHoplitis villosa (Schenck, ...",paragraphs,"```json\n{\n ""meta_location"": ""Berlin, Dahlem...",https://www.zobodat.at/pdf/Maerkische-Ent-Nach...,True,train
2,Beitrag zur Hautflüglerfauna von Bran-\ndenbur...,paragraphs,"```json\n{\n ""meta_scientific_name"": ""Andrena...",https://www.zobodat.at/pdf/Maerkische-Ent-Nach...,True,train
...,...,...,...,...,...,...
27,Für das Stadtgebiet Bielefeld nördlich des Te...,list_table,"```json\n{\n ""meta_location"": ""Nordrhein-West...",https://www.zobodat.at/pdf/Ber-Natwiss-Ver-Bie...,True,eval
28,Für das Stadtgebiet Bielefeld nördlich des Te...,list_table,"```json\n{\n ""meta_location"": ""Nordrhein-West...",https://www.zobodat.at/pdf/Ber-Natwiss-Ver-Bie...,True,eval
29,Die Insekten des Naturschutzparkes\nder Lünebu...,list_table,"```json\n{\n ""meta_location"": ""Lüneburger Hei...",https://www.zobodat.at/pdf/Abh-natwiss-Verein-...,True,eval


In [7]:
datah=Datahandle(
    label='beede_llm_dataset',
    data_df=dataset_df,
)
datah.config
datah.save()

In [8]:
reimport()

In [11]:
experiment=ExtractionExperiment(
    llm_pipe=LlamaPipeline(),
    data_df=Datahandle(1),

    extraction_pipe=ExtractionPipeline(
        use_few_shots = True,
        use_extraction_prompt = False
    ),

    num_train_epochs=2,
    save_strategy='epoch',
    save_total_limit=2,

    do_eval_base_model=True,
    do_eval_lora_model=True,
    do_train=True
    )

experiment.config
experiment.save()

In [16]:
experiments_df=FinetuningExperiment().get_overview(keypaths=['done','num_train_epochs','do_eval_base_model','do_train','_path'])
experiments_df

,class_parts,id,label,done,num_train_epochs,do_eval_base_model,do_train,_path
0,"[FinetuningExperiment, ExtractionExperiment]",1,ExtractionExperiment,False,2,True,True,/home/fboehning/fboehning/extraction_lab/Finet...


In [17]:
experiments_df.iloc[0]._path

'/home/fboehning/fboehning/extraction_lab/FinetuningExperiment/ExtractionExperiment/0001_ExtractionExperiment'